Instalamos las librerías que nos permiten usar *SQL* con *pandas*.

In [ ]:
!pip install ipython-sql
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importamos librerías y utilizamos funciones lambda para facilitar el proceso de usar *SQL*.

In [ ]:
from pandasql import sqldf
import numpy as np
import pandas as pd
pysql = lambda q: sqldf(q, globals())

Cargamos la tabla encontrada en el [link](https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations), por facilidad vamos a cambiar el nombre de algunas columnas.

In [ ]:
url_tabla = 'https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)'
df_paises = pd.read_html(url_tabla)[0]
df_paises = df_paises.rename(columns={'Country / Area':'Country','UN continentalregion[4]':'Region','UN statisticalsubregion[4]':'Subregion','Population(1 July 2018)':'Population_2018','Population(1 July 2019)':'Population_2019'})
df_paises

,Country,Region,Subregion,Population_2018,Population_2019,Change
0,China[a],Asia,Eastern Asia,1427647786,1433783686,+0.43%
1,India,Asia,Southern Asia,1352642280,1366417754,+1.02%
2,United States,Americas,Northern America,327096265,329064917,+0.60%
3,Indonesia,Asia,South-eastern Asia,267670543,270625568,+1.10%
4,Pakistan,Asia,Southern Asia,212228286,216565318,+2.04%
...,...,...,...,...,...,...
229,Falkland Islands (United Kingdom),Americas,South America,3234,3377,+4.42%
230,Niue,Oceania,Polynesia,1620,1615,−0.31%
231,Tokelau (New Zealand),Oceania,Polynesia,1319,1340,+1.59%
232,Vatican City[z],Europe,Southern Europe,801,799,−0.25%


Eliminamos la ultima fila, que corresponde a las estadísticas del mundo.

In [ ]:
df_paises = df_paises.drop(index=233)
df_paises

,Country,Region,Subregion,Population_2018,Population_2019,Change
0,China[a],Asia,Eastern Asia,1427647786,1433783686,+0.43%
1,India,Asia,Southern Asia,1352642280,1366417754,+1.02%
2,United States,Americas,Northern America,327096265,329064917,+0.60%
3,Indonesia,Asia,South-eastern Asia,267670543,270625568,+1.10%
4,Pakistan,Asia,Southern Asia,212228286,216565318,+2.04%
...,...,...,...,...,...,...
228,Montserrat (United Kingdom),Americas,Caribbean,4993,4989,−0.08%
229,Falkland Islands (United Kingdom),Americas,South America,3234,3377,+4.42%
230,Niue,Oceania,Polynesia,1620,1615,−0.31%
231,Tokelau (New Zealand),Oceania,Polynesia,1319,1340,+1.59%


Ahora vamos a convertir el atributo *Change* a uno númerico, ya que este es un string y esto dificulta el análisis.

In [ ]:
m1 = df_paises['Change'].str.startswith('−')  
m2 = df_paises['Change'].str[1:-1].astype(float)  
df_paises['Change'] = np.where(m1, -m2, m2)


<h3>Mayor población durante 2019</h3>

Veamos el top 5 de países con mayor población en 2019.

In [ ]:
consulta = '''SELECT Country, Population_2019
FROM df_paises
ORDER BY Population_2019 DESC 
LIMIT 5;'''
pysql(consulta)

,Country,Population_2019
0,China[a],1433783686
1,India,1366417754
2,United States,329064917
3,Indonesia,270625568
4,Pakistan,216565318


El país con mayor población durante el 2019 es China.

<h3>Mayor tasa de crecimiento para el periodo 2018-2019</h3>


Veamos el top 5 de países con mayor tasa de crecimiento para el periodo 2018-2019. Recordemos que esta tasa se encuentra en el atributo *Change*

In [ ]:
consulta = '''SELECT Country, Change 
FROM df_paises
ORDER BY Change desc 
LIMIT 5;'''
pysql(consulta)

,Country,Change
0,Bahrain,4.57
1,Falkland Islands (United Kingdom),4.42
2,Niger,3.87
3,Uganda,3.61
4,Equatorial Guinea,3.59


El país con mayor tasa de crecimiento es Bahrain(Baréin).

<h3>Menor tasa de crecimiento para el periodo 2018-2019</h3>


Veamos el top 5 de países con menor tasa de crecimiento para el periodo 2018-2019.

In [ ]:
consulta = '''SELECT Country, Change 
FROM df_paises
WHERE Change IS NOT NULL
ORDER BY Change asc 
LIMIT 5;'''
pysql(consulta)

,Country,Change
0,Puerto Rico (United States),-3.49
1,Wallis and Futuna (France),-1.96
2,Lithuania,-1.49
3,Venezuela,-1.29
4,Northern Mariana Islands (United States),-1.22


El país con menor tasa de crecimiento es Puerto Rico. Note que usamos el comando *WHERE* para filtrar países que no tengan datos registrados en el atributo *Change*. Veamos que ocurre si no realizamos el filtro

In [ ]:
consulta = '''SELECT Country, Change 
FROM df_paises
ORDER BY Change asc 
LIMIT 5;'''
pysql(consulta)

,Country,Change
0,Belarus,NaN
1,Puerto Rico (United States),-3.49
2,Wallis and Futuna (France),-1.96
3,Lithuania,-1.49
4,Venezuela,-1.29


<h3>Tasa de crecimiento de Asia</h3>

Asumimos que se está buscando el promedio, ya que es necesario agrupar los datos y combinarlos de alguna forma.

In [ ]:
consulta = '''SELECT Region,AVG(Change) as Promedio_Tasa
FROM df_paises
WHERE Region='Asia'
GROUP BY Region;'''
pysql(consulta)

,Region,Promedio_Tasa
0,Asia,1.338039


Vemos que el promedio de tasa de crecimiento para Asia es *1.33%*

<h3>Tasa de crecimiento de America del sur</h3>

Debemos agrupar los datos mediante *Region* y *Subregion*, y luego calculamos el promedio.

In [ ]:
consulta = '''SELECT Region,Subregion,AVG(Change) as Promedio_Tasa
FROM df_paises
WHERE Region='Americas' AND Subregion='South America'
GROUP BY Region,Subregion;'''
pysql(consulta)

,Region,Subregion,Promedio_Tasa
0,Americas,South America,1.265


Vemos que el promedio de tasa de crecimiento para America del sur es de *1.26%*

<h3>Continentes con mayor tasa</h3>

De forma similar a las consultas anteriores, agrupamos por *Region* y ahora organizamos de mayor a menor la tasa de crecimiento.

In [ ]:
consulta = '''SELECT Region,AVG(Change) as Promedio_Tasa
FROM df_paises
GROUP BY Region
ORDER BY Promedio_Tasa DESC'''
pysql(consulta)

,Region,Promedio_Tasa
0,Africa,2.251207
1,Asia,1.338039
2,Americas,0.768302
3,Oceania,0.734348
4,Europe,0.120426


El continente con mayor tasa de crecimiento es África.

<h3>Continentes con menor tasa</h3>

De forma análoga a la consulta anterior, solo que esta vez organizamos de menor a mayor.

In [ ]:
consulta = '''SELECT Region,AVG(Change) as Promedio_Tasa
FROM df_paises
GROUP BY Region
ORDER BY Promedio_Tasa ASC'''
pysql(consulta)

,Region,Promedio_Tasa
0,Europe,0.120426
1,Oceania,0.734348
2,Americas,0.768302
3,Asia,1.338039
4,Africa,2.251207


El continente con menor tasa de crecimiento es Europa.